In [1]:
import json

with open('performance_exchange.json') as jsonFile:
    exchanges = json.load(jsonFile)
    jsonFile.close()

In [2]:
len(exchanges)

121

In [70]:
import ccxt
import os
import sys
from pprint import pprint
import random
import requests
import datetime
import json


time = '06/08/2022 06:11'

def get_crypto_price(time, name, crypto_id, crypto_symbol, stop_rate = 0.2):
    '''
    Finds the price of the crypto (based on USDT) at a exac
    t time (1 minute precission).
    It search the information in several exanges. 
    To go quicker reduce de number of exqnges to check with stop_rate.
    If price not found, it searches the information of the daily price (not 1 minute precission) in coingecko.
    '''
    
    currency = crypto_symbol + '/USDT'

    if isinstance(time, datetime.date): time = time.strftime("%d/%m/%Y %H:%M")
    timestamp = int(datetime.datetime.strptime(time, '%d/%m/%Y %H:%M').timestamp() * 1000)

    i = 0
    found = 0
    
    with open('performance_exchange.json') as jsonFile:
        performance_exchange = json.load(jsonFile)
        jsonFile.close()

    #exchanges = list(ccxt.exchanges)
    #exchanges = random.sample(exchanges, len(exchanges))
    
    stop = False
    stop_at = int(stop_rate * len(performance_exchange))
    
    price_exchange = {}
    #performance_exchange = {}
    
    # This for loop tries to find the currency in one exange
    for exchange_id in performance_exchange:
        i += 1
        try:
            exchange = getattr(ccxt, exchange_id)()
            exchange.load_markets()

            if currency.upper() in exchange.symbols:

                price = exchange.fetch_ohlcv(currency.upper(), '1m', timestamp, 1)[0][-2]

                if isinstance(price,float):
                    price_exchange[exchange_id] = price
                    print(currency.upper(), time, exchange_id, price)
                    found += 1

                if found >= 3:
                    ratios = {}
                    for ex in price_exchange:

                        ratios[ex] = []
                        for ex2 in price_exchange:
                            ratios[ex].append(abs((price_exchange[ex] - price_exchange[ex2]) / price_exchange[ex]) >= 0.01)

                    p_e = {}
                    for ex in ratios:
                        if sum(ratios[ex]) < len(ratios[ex])/2:
                            p_e[ex] = price_exchange[ex]

                    if p_e == {}:
                        found = 0
                    else:
                        price_exchange = p_e
            else:
                print(currency.upper(), 'NOT FOUND in ', exchange_id)
                
            if i > stop_at:
                stop = True
                break

            if (found >= 3) or (i >=len(performance_exchange)):
                for ex in price_exchange:
                    performance_exchange[ex] += 1

                performance_exchange = {k: v for k, v in sorted(performance_exchange.items(), key=lambda item: item[1], reverse=True)}
                with open('performance_exchange.json', 'w') as f:
                    json.dump(performance_exchange, f)

                prices = []
                for exchange in price_exchange:
                    prices.append(price_exchange[exchange])

                price = sum(prices) / len(prices)
                print('price', currency.upper(), time, price)

                #print(price_exchange, performance_exchange)
                #return price_exchange, performance_exchange, ratios
                return price
                break

        except:
            print(currency.upper(), 'NOT FOUND in ', exchange_id)

    if stop:

        try:
            url = 'https://api.coingecko.com/api/v3/coins/' + crypto_id + '/history?date=' + time[:10].replace('/','-')
                        
            data = requests.get(url)
            price = data.json()
            
            return price['market_data']['current_price']['usd'], 'coingecko'
        except:
            price = float(input('Enter price {} {} manually'.format(currency.upper(), time)))
            
            return price

In [71]:
price = get_crypto_price(time, 'Bitcoin', 'bitcoin', 'BTC')

BTC/USDT 06/08/2022 06:11 lbank2 23181.93
BTC/USDT 06/08/2022 06:11 huobipro 23186.08
BTC/USDT 06/08/2022 06:11 hollaex 23183.25
price BTC/USDT 06/08/2022 06:11 23183.753333333338


In [72]:
#price = get_crypto_price(time, 'sd', 'af', 'XDD')

In [73]:
class Portfolio():
    def __init__(self, name = 'Portfolio'):
        '''
        Initialisation of the Portfolio.
        '''
        self.name = name.lower()
        self.wallets = {}
        
    def add_wallet(self, wallet):
        '''
        Add wallet to the portfolio
        '''
        self.wallets[wallet.chain]=wallet

In [74]:
pf = Portfolio()

In [75]:
class Wallet():
    def __init__(self, chain='Polygon', portfolio=''):
        '''
        Initialisation of the wallet chain. It stores all crytos in the same chain.
        '''
        self.cryptos = {}
        self.chain = chain.lower()
        self.name = 'wallet_' + chain.lower()
        portfolio.add_wallet(self)
    
    def add_crypto(self, crypto, main_crypto_chain=False):
        '''
        Add a crypto to the wallet
        '''
        self.cryptos[crypto.name] = crypto
        
        if main_crypto_chain:
            self.main_crypto_chain = crypto
            
    def get_info(self):
        '''
        Show state of the wallet
        '''
        self.info = {}
        for coin in self.cryptos:
            self.cryptos[coin].get_info()
            self.info[coin] = self.cryptos[coin].info

In [76]:
wallet_poly = Wallet('Polygon', pf)

In [77]:
# Import libraries
import json


class Crypto():
    def __init__(self, name = '', symbol = '', quantity=0, time='', price=-1, btc_price=-1, wallet='',main_crypto_chain=False):
        '''
        Initialisation of the cryptocurrency: gets the price and symbol
        '''
        self.name = name.lower()
        self.symbol = symbol.lower()
        self.chain = wallet.chain
        self.location = wallet.name
        self.wallet = wallet
        self.counter = 0
        self.fees = 0
        self.fees_cumulated = 0
        self.fees_cumulated_value = 0
        
        self.get_categories()
        
        self.get_now(quantity, price, time, btc_price, initial=True)
        
        wallet.add_crypto(self,main_crypto_chain)
    
    
    def update_quantity(self,quantity):
        '''
        Updates the quantity of the crypto
        '''
        self.get_now(quantity=quantity, price=self.price_now, time=self._time_now, btc_price=self.btc_price_now, purchassed=False )
        
    def buy(self,quantity, time=''):
        '''
        Buy more
        '''
        quantity = self.quantity_now + quantity
        self.get_now(quantity=quantity, purchassed=True, action = 'buy', time = time)
    
    def get_symbol(self):
        '''
        gets the symbol of the crypto
        '''
        url = 'https://api.coingecko.com/api/v3/coins/'
        data = requests.get(url + 'list')
        coins_list = data.json()
        
        for crypto in coins_list:
            try:
                if (self.symbol != '') and (crypto['symbol'] == self.symbol):
                    return crypto['id'], crypto['symbol']
                elif crypto['name'].lower() == self.name:
                    return crypto['id'], crypto['symbol']
            except:
                if self.symbol == '':
                    self.symbol = input("Enter symbol manually").lower()
                if self.name == '':
                    self.name = input("Enter name manually").lower()
                return self.name, self.symbol
    
    def get_categories(self):
        
        self.id, self.symbol = self.get_symbol()
        
        url = 'https://api.coingecko.com/api/v3/coins/'
        
        try:
            data = requests.get(url + self.id)
            self.categories = data.json()['categories']
        except:
            print(requests.get(url + self.id), 'did not work')
            self.categories = input("Enter category manually").lower()
        
    def get_now(self, quantity=0, price=-1, time='', btc_price=-1, initial=False, action='', purchassed=True):
        '''
        Add a quantity of tokens manually
        '''
        
        self.purchassed = purchassed
        self.get_time_now(time, initial)
        
        self.get_price_now(price, time, initial)
        
        if initial:
            self.get_token_proportion(1)
            self.quantity_now = 0
            self.get_identifier()
        
        self.get_quantity_now(quantity, initial)
        self.get_quantity_delta()
        
        self.get_price_mean(initial, purchassed)
            
        self.get_value_now(initial)
        self.get_value_mean()
        
        self.get_btc_price_now(btc_price, time, initial)
        self.get_btc_price_mean(initial, purchassed)
        self.get_btc_value_now(initial)
        self.get_btc_value_mean()
        
        self.get_return()
        self.get_btc_return()
        
        self.get_info()
        self.get_trace(action, quantity, initial)
        
        self.quantity_old = self.quantity_now
        
        self.counter += 1
    
    def get_fees(self, fees=0):
        '''
        This method is used to update crypto quantity when fees are applied
        '''
        self.get_now(quantity=self.quantity_now - fees, action='fees', purchassed=False)
        self.fees = fees
        self.fees_cumulated = self.fees_cumulated + fees
        self.fees_cumulated_value = self.fees_cumulated * self.price_now
        
    def close(self):
        '''
        Turns quantity to 0
        '''
        self.quanity_now = 0
        self.get_now()
    
    def get_time_now(self, time='', initial=False):
        '''
        Gets actual time
        '''
        if time == '':
            time = datetime.datetime.now()
        else:
            if not isinstance(time, datetime.date): time = datetime.datetime.strptime(time, '%d/%m/%Y %H:%M')
            self.time_now = time
        
        if initial:
            self.time_buy = self.time_now
            
        self.time_delta_buy = self.time_now - self.time_buy
        
    def update_chain(self, chain):
        '''
        Updates chaint token
        '''
        self.chain = chain
        
    def get_identifier(self, identifier=''):
        '''
        Gets an identifier for this crypto
        '''
        if identifier != '':
            self.identifier = identifier
        else:
            t = self.time_now
            self.identifier = self.symbol + '_in_' + self.chain + '_' + self.location + '_' + str(t.year) + '_' + str(t.month) + '_' + str(t.day) + '_' + str(t.hour) + '_' + str(t.minute)
    
    def get_price_any(self, time='', name='Bitcoin', ident='bitcoin', symbol='btc', stop_rate=1):
        '''
        Gets price of any crypto at any time
        '''
        if time == '': time = self.time_now
         
        return float(get_crypto_price(time, name, ident, symbol, stop_rate))
    
    def get_price_now(self, price=-1, time='', initial=False):
        '''
        Gets actual price of the crypto at now time
        '''
        # keep old value
        if initial:
            self.price_old = 0
        else:
            self.price_old = self.price_now
                
        if price == -1:
            self.price_now = self.get_price_any(time, self.name, self.id, self.symbol, 0.2)
        else:
            self.price_now = price
            
        if initial: self.price_buy = self.price_now
    
    def get_price_mean(self, initial=False, purchassed=True, price=-1):
        '''
        Calculates the mean price of the crypto
        '''
        # Keep preivous mean price
        if initial:
            self.price_mean_old = 0
            self.price_mean = 0
        else:
            self.price_mean_old = self.price_mean
        
        if price == -1:
            # Only updates mean price if new tokens are purchassed (instead of update due to LP or fees)
            if purchassed:
                self.price_mean = round((self.price_mean_old * self.quantity_old + self.price_now * self.quantity_delta) / self.quantity_now, 7)
        else:
            self.price_mean = price
                
    def get_quantity_now(self, quantity, initial=False):
        '''
        Updates quantity of now tokens manually
        '''
        # keep old value
        if initial:
            self.quantity_old = 0
            self.quantity_buy = self.quantity_now
        else:
            self.quantity_old = self.quantity_now
        
        if quantity != 0: self.quantity_now = quantity
            
    def get_quantity_delta(self):
        '''
        Updates delta quantity of now and old tokens
        '''
        self.quantity_delta =  self.quantity_now - self.quantity_old
    
    def get_value_now(self, initial = False):
        '''
        Calculates the value of the Crypto at now price
        '''
        self.value_now = round(self.quantity_now * self.price_now, 7)
        
        if initial: self.value_buy = self.value_now
        
    def get_value_mean(self):
        '''
        Calculates the value of the Crypto at mean price
        '''
        self.value_mean = self.quantity_now * self.price_mean

    def get_btc_price_now(self, price=-1, time='', initial=False):
        '''
        Gets the price of BTC
        '''
        # Keep old BTC price
        if initial:
            self.btc_price_old = 0
        else:
            self.btc_price_old = self.btc_price_now
        
        if price == -1:
            self.btc_price_now = self.get_price_any(time, 'Bitcoin', 'bitcoin', 'btc')
        else:
            self.btc_price_now = price
        
        if initial: self.btc_price_buy = self.btc_price_now
        
    def get_btc_price_mean(self, initial=False, purchassed=True, price=-1):
        '''
        Calculates the mean price of the BTC
        '''
         # Keep previous mean BTC price
        if initial:
            self.btc_price_mean_old = 0
        else:
            self.btc_price_mean_old = self.btc_price_mean
        
        if price == -1:
            if purchassed:
                self.btc_price_mean = round((self.btc_price_mean_old * self.quantity_old + self.btc_price_now * self.quantity_delta) / self.quantity_now, 7)
        else:
            self.btc_price_mean = price
        
    def get_btc_value_now(self, initial=False):
        '''
        Gets the value of the tokens in BTC at now time
        '''
        # Get value in BTC
        self.btc_value_now = round(self.value_now / self.btc_price_now, 7)
        
        if initial:
            self.btc_value_old = 0
            self.btc_value_buy = self.btc_value_now
    
    def get_btc_value_mean(self):
        '''
        Calculates the value of the BTC at mean price
        '''
        self.btc_value_mean = round(self.value_mean / self.btc_price_mean, 7)
    
    def get_token_proportion(self, proportion):
        '''
        Stores the proportion of the token. Updated via LiquidityPool class
        '''
        self.token_proportion = proportion
        
    def get_return(self):
        '''
        Calculates the return of the crypto regarding value_mean
        '''
        self.returns = round((self.value_now - self.value_mean) / self.value_mean * 100, 1)
        
    def get_btc_return(self):
        '''
        Calculates the delta value of the crypto
        '''
        self.btc_return = round((self.btc_value_now - self.btc_value_mean) / self.btc_value_mean * 100, 1)
    
    def get_info(self):
        '''
        Outputs a dictionary of all values inside the class
        '''
        dict_info = {
            'purchassed': self.purchassed,
            'time_buy': self.time_buy,
            'time_now': self.time_now,
            'time_delta': self.time_delta_buy,
            
            'price_buy': self.price_buy,
            'price_old': self.price_old,
            'price_now': self.price_now,
            'price_mean_old': self.price_mean_old,
            'price_mean': self.price_mean,
            
            'fees': self.fees,
            'fees_cumulated': self.fees_cumulated,
            'quantity_buy': self.quantity_buy,
            'quantity_old': self.quantity_old,
            'quantity_now': self.quantity_now,
            'quantity_delta': self.quantity_delta,
            
            'value_buy': self.value_buy,
            'value_now': self.value_now,
            'value_mean': self.value_mean,
            'fees_cumulated_value': self.fees_cumulated_value,
            
            'btc_price_buy': self.btc_price_buy,
            'btc_price_old': self.btc_price_old,
            'btc_price_now': self.btc_price_now,
            'btc_price_mean': self.btc_price_mean,
            
            'btc_value_buy': self.btc_value_buy,
            'btc_value_now': self.btc_value_now,
            'btc_value_mean': self.btc_value_mean,
            
            'token_proportion': self.token_proportion,
            
            'return %': self.returns,
            'btc_return %': self.btc_return
        }
        self.info = dict_info
    
    def get_trace(self, action='', quantity=0, initial=False):
        '''
        Keeps trace of the evolution of the crypto
        '''
        if action != '':
            action = action
        elif quantity != 0:
            action = 'add_quantity'
        else:
            action = 'crypto_update'
        
        if initial:
            self.trace = {}
            self.trace[self.identifier] = {}
            self.trace[self.identifier]['id'] = self.id
            self.trace[self.identifier]['name'] = self.name
            self.trace[self.identifier]['categories'] = self.categories
            self.trace[self.identifier]['chain'] = self.chain
            self.trace[self.identifier]['location'] = self.location
            self.trace[self.identifier]['symbol'] = self.symbol
            action = 'crypto_initial'
            
        self.timestamp = self.time_now.strftime("%Y/%m/%d %H:%M:%S")
        
        self.trace[self.identifier][self.counter] = {}
        self.trace[self.identifier][self.counter]['state'] = self.info
        self.trace[self.identifier][self.counter]['action'] = action
        
        

In [78]:
btc = Crypto('Bitcoin',quantity = 0.001, time='11/03/2022 12:00', wallet = wallet_poly)
#btc.get_now(quantity = 0.002)
#btc.get_now(quantity = 0.003)
#btc.get_now(quantity = 0.003)
#btc.get_now()
btc.trace

BTC/USDT 11/03/2022 12:00 lbank2 39008.53
BTC/USDT 11/03/2022 12:00 huobipro 38995.7
BTC/USDT 11/03/2022 12:00 hollaex 39025.85
price BTC/USDT 11/03/2022 12:00 39010.026666666665
BTC/USDT 11/03/2022 12:00 lbank2 39008.53
BTC/USDT 11/03/2022 12:00 huobipro 38995.7
BTC/USDT 11/03/2022 12:00 hollaex 39025.85
price BTC/USDT 11/03/2022 12:00 39010.026666666665


{'btc_in_polygon_wallet_polygon_2022_3_11_12_0': {'id': 'bitcoin',
  'name': 'bitcoin',
  'categories': ['Cryptocurrency', 'Layer 1 (L1)'],
  'chain': 'polygon',
  'location': 'wallet_polygon',
  'symbol': 'btc',
  0: {'state': {'purchassed': True,
    'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'time_delta': datetime.timedelta(0),
    'price_buy': 39010.026666666665,
    'price_old': 0,
    'price_now': 39010.026666666665,
    'price_mean_old': 0,
    'price_mean': 39010.0266667,
    'fees': 0,
    'fees_cumulated': 0,
    'quantity_buy': 0,
    'quantity_old': 0,
    'quantity_now': 0.001,
    'quantity_delta': 0.001,
    'value_buy': 39.0100267,
    'value_now': 39.0100267,
    'value_mean': 39.01002666670001,
    'fees_cumulated_value': 0,
    'btc_price_buy': 39010.026666666665,
    'btc_price_old': 0,
    'btc_price_now': 39010.026666666665,
    'btc_price_mean': 39010.0266667,
    'btc_value_buy': 0.001,
    'btc_

In [79]:
btc.buy(quantity = 0.0001, time='02/03/2023 12:00')
btc.trace

BTC/USDT NOT FOUND in  lbank2
BTC/USDT NOT FOUND in  huobipro
BTC/USDT NOT FOUND in  hollaex
BTC/USDT NOT FOUND in  bitmart
BTC/USDT NOT FOUND in  timex
BTC/USDT NOT FOUND in  gate
BTC/USDT NOT FOUND in  digifinex
BTC/USDT NOT FOUND in  gateio
BTC/USDT NOT FOUND in  zb
BTC/USDT NOT FOUND in  bequant
BTC/USDT NOT FOUND in  aax
BTC/USDT NOT FOUND in  bytetrade
BTC/USDT NOT FOUND in  ascendex
BTC/USDT NOT FOUND in  kucoin
BTC/USDT NOT FOUND in  eqonex
BTC/USDT NOT FOUND in  binancecoinm
BTC/USDT NOT FOUND in  bitvavo
BTC/USDT NOT FOUND in  coinmate
BTC/USDT NOT FOUND in  coinbase
BTC/USDT NOT FOUND in  qtrade
BTC/USDT NOT FOUND in  zipmex
BTC/USDT NOT FOUND in  blockchaincom
BTC/USDT NOT FOUND in  ndax
BTC/USDT NOT FOUND in  itbit
BTC/USDT NOT FOUND in  mexc3
BTC/USDT NOT FOUND in  bl3p


Enter price BTC/USDT 11/03/2023 12:00 manually 22000


BTC/USDT NOT FOUND in  lbank2
BTC/USDT NOT FOUND in  huobipro
BTC/USDT NOT FOUND in  hollaex
BTC/USDT NOT FOUND in  bitmart
BTC/USDT NOT FOUND in  timex
BTC/USDT NOT FOUND in  gate
BTC/USDT NOT FOUND in  digifinex
BTC/USDT NOT FOUND in  gateio
BTC/USDT NOT FOUND in  zb
BTC/USDT NOT FOUND in  bequant
BTC/USDT NOT FOUND in  aax
BTC/USDT NOT FOUND in  bytetrade
BTC/USDT NOT FOUND in  ascendex
BTC/USDT NOT FOUND in  kucoin
BTC/USDT NOT FOUND in  eqonex
BTC/USDT NOT FOUND in  binancecoinm
BTC/USDT NOT FOUND in  bitvavo
BTC/USDT NOT FOUND in  coinmate
BTC/USDT NOT FOUND in  coinbase
BTC/USDT NOT FOUND in  qtrade
BTC/USDT NOT FOUND in  zipmex
BTC/USDT NOT FOUND in  blockchaincom
BTC/USDT NOT FOUND in  ndax
BTC/USDT NOT FOUND in  itbit
BTC/USDT NOT FOUND in  mexc3
BTC/USDT NOT FOUND in  bl3p
BTC/USDT NOT FOUND in  fmfwio
BTC/USDT NOT FOUND in  coinbaseprime
BTC/USDT NOT FOUND in  delta
BTC/USDT NOT FOUND in  binance
BTC/USDT NOT FOUND in  bw
BTC/USDT NOT FOUND in  bitbns
BTC/USDT NOT FOUND in 

{'btc_in_polygon_wallet_polygon_2022_3_11_12_0': {'id': 'bitcoin',
  'name': 'bitcoin',
  'categories': ['Cryptocurrency', 'Layer 1 (L1)'],
  'chain': 'polygon',
  'location': 'wallet_polygon',
  'symbol': 'btc',
  0: {'state': {'purchassed': True,
    'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'time_delta': datetime.timedelta(0),
    'price_buy': 39010.026666666665,
    'price_old': 0,
    'price_now': 39010.026666666665,
    'price_mean_old': 0,
    'price_mean': 39010.0266667,
    'fees': 0,
    'fees_cumulated': 0,
    'quantity_buy': 0,
    'quantity_old': 0,
    'quantity_now': 0.001,
    'quantity_delta': 0.001,
    'value_buy': 39.0100267,
    'value_now': 39.0100267,
    'value_mean': 39.01002666670001,
    'fees_cumulated_value': 0,
    'btc_price_buy': 39010.026666666665,
    'btc_price_old': 0,
    'btc_price_now': 39010.026666666665,
    'btc_price_mean': 39010.0266667,
    'btc_value_buy': 0.001,
    'btc_

In [ ]:
test = Crypto(
    name = 'Wrapped Bitcoin', 
    symbol = 'wbtc',
    quantity=0.043, 
    time='11/03/2022 12:00', 
    price=23400, 
    btc_price=23400, 
    wallet=wallet_poly,
    main_crypto_chain=False
)

In [14]:
vars(wallet_poly)

{'cryptos': {'bitcoin': <__main__.Crypto at 0x1b702c584f0>,
  'wrapped bitcoin': <__main__.Crypto at 0x1b7019aac10>},
 'chain': 'polygon',
 'name': 'wallet_polygon'}

In [15]:
eth = Crypto('Ethereum',quantity = 0.0232, time='11/03/2022 12:00', wallet = wallet_poly)
eth.trace

ETH/USDT 11/03/2022 12:00 lbank2 2599.27
ETH/USDT 11/03/2022 12:00 huobipro 2599.1
ETH/USDT 11/03/2022 12:00 hollaex 2594.1
price ETH/USDT 11/03/2022 12:00 2597.49
BTC/USDT 11/03/2022 12:00 lbank2 39008.53
BTC/USDT 11/03/2022 12:00 huobipro 38995.7
BTC/USDT 11/03/2022 12:00 hollaex 39025.85
price BTC/USDT 11/03/2022 12:00 39010.026666666665


{'eth_in_polygon_wallet_polygon_2022_3_11_12_0': {'id': 'ethereum',
  'name': 'ethereum',
  'categories': ['Smart Contract Platform',
   'Ethereum Ecosystem',
   'Layer 1 (L1)'],
  'chain': 'polygon',
  'location': 'wallet_polygon',
  'symbol': 'eth',
  0: {'state': {'purchassed': True,
    'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'time_delta': datetime.timedelta(0),
    'price_old': 0,
    'price_now': 2597.49,
    'price_mean_old': 0,
    'price_mean': 2597.49,
    'fees': 0,
    'fees_cumulated': 0,
    'quantity_old': 0,
    'quantity_now': 0.0232,
    'quantity_delta': 0.0232,
    'value_now': 60.261768,
    'value_mean': 60.26176799999999,
    'fees_cumulated_value': 0,
    'btc_price_old': 0,
    'btc_price_now': 39010.026666666665,
    'btc_price_mean': 39010.0266667,
    'btc_value_now': 0.0015448,
    'btc_value_mean': 0.0015448,
    'token_proportion': 1,
    'return %': 0.0,
    'btc_return %': 0.0},
   'a

In [16]:
matic = Crypto('Polygon', symbol = 'MATIC',quantity = 30, time='11/03/2022 12:00', wallet = wallet_poly, main_crypto_chain=True)
matic.trace

MATIC/USDT 11/03/2022 12:00 lbank2 1.4456
MATIC/USDT 11/03/2022 12:00 huobipro 1.43884
MATIC/USDT NOT FOUND in  hollaex
MATIC/USDT 11/03/2022 12:00 bitmart 1.44
price MATIC/USDT 11/03/2022 12:00 1.4414799999999997
BTC/USDT 11/03/2022 12:00 lbank2 39008.53
BTC/USDT 11/03/2022 12:00 huobipro 38995.7
BTC/USDT 11/03/2022 12:00 hollaex 39025.85
price BTC/USDT 11/03/2022 12:00 39010.026666666665


{'matic_in_polygon_wallet_polygon_2022_3_11_12_0': {'id': 'matic-network',
  'name': 'polygon',
  'categories': ['Harmony Ecosystem',
   'Polygon Ecosystem',
   'Moonbeam Ecosystem',
   'Zero Knowledge (ZK)',
   'Moonriver Ecosystem',
   'Smart Contract Platform',
   'BNB Chain Ecosystem',
   'Ethereum Ecosystem',
   'Layer 2 (L2)'],
  'chain': 'polygon',
  'location': 'wallet_polygon',
  'symbol': 'matic',
  0: {'state': {'purchassed': True,
    'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'time_delta': datetime.timedelta(0),
    'price_old': 0,
    'price_now': 1.4414799999999997,
    'price_mean_old': 0,
    'price_mean': 1.44148,
    'fees': 0,
    'fees_cumulated': 0,
    'quantity_old': 0,
    'quantity_now': 30,
    'quantity_delta': 30,
    'value_now': 43.2444,
    'value_mean': 43.244400000000006,
    'fees_cumulated_value': 0,
    'btc_price_old': 0,
    'btc_price_now': 39010.026666666665,
    'btc_price_mean'

In [17]:
wallet_arbitrum = Wallet('Arbitrum', pf)

In [18]:
vars(wallet_poly)

{'cryptos': {'bitcoin': <__main__.Crypto at 0x1b702c584f0>,
  'wrapped bitcoin': <__main__.Crypto at 0x1b7019aac10>,
  'ethereum': <__main__.Crypto at 0x1b7019cbcd0>,
  'polygon': <__main__.Crypto at 0x1b702c50d00>},
 'chain': 'polygon',
 'name': 'wallet_polygon',
 'main_crypto_chain': <__main__.Crypto at 0x1b702c50d00>}

In [19]:
vars(pf)

{'name': 'portfolio',
 'wallets': {'polygon': <__main__.Wallet at 0x1b702329c10>,
  'arbitrum': <__main__.Wallet at 0x1b701eb04c0>}}

In [20]:
from math import sqrt
import numpy as np
from sympy.solvers import solve
from sympy import symbols, solve_poly_system
import sympy

class LiquidityPool():
    def __init__(self, cryptos, APY=0, time=''):
        '''
        Initialisation of the liquidity pool
        '''
        self.update_APY(APY)
        self.cryptos = cryptos
        self.counter = 0
        
        self.get_now('creation', initial = True, time = time)
    
    def get_now(self, action = 'update', initial=False, manual=False, time=''):
        '''
        Updates all the variables at now time
        '''
        self.action = action
        if manual: self.action = self.action + '_manual'
        
        self.get_time_now(initial, time)
        self.get_k(initial)
        self.get_price_now(initial)
        self.get_delta_time(initial)
        
        self.get_quantity_APY_return(initial, manual)
        self.get_quantity_now(initial, manual)
        self.udpdate_cryptos_quantities(initial)
        
        self.get_value_now(initial)
        self.get_btc_value_now(initial)
        
        self.get_impermanent_loss(initial) 
        self.get_impermanent_loss_percentage(initial)
        self.get_return_total(initial)
        
        self.get_tokens_proportion(initial)
        self.get_identifier(initial)
        self.get_info()
        self.get_trace(initial)
        self.counter += 1
        
    def get_time_now(self, initial=False, time=''):
        '''
        Gets actual time
        '''        
        if time == '':
            self.time_now = datetime.datetime.now()
        else:
            if not isinstance(time, datetime.date): time = datetime.datetime.strptime(time, '%d/%m/%Y %H:%M')
            self.time_now = time
            
        if initial: 
            self.time_buy = self.time_now
            self.last_time = self.time_now
            self.last_claim_time = self.time_now
    
    def get_k(self, initial=False):
        '''
        Gets the value of the invariant K => q1·q2·q3...=K at the begininig of the liquidity pool. 
        Where q1, q2, q3... are the token quantity of each crypto.
        '''
        if initial:
            self.k = 1
            for crypto in self.cryptos:
                self.k = crypto.quantity_now * self.k
        else:
            self.k = 1
            for quantity_now in self.quantity_now:
                self.k = quantity_now * self.k
        
    def get_price_now(self, initial=False):
        '''
        Updates all cryptos prices
        '''
        self.tokens_price_now = []
        for crypto in self.cryptos:
            self.tokens_price_now.append(crypto.price_now)
            
        if initial:
            self.tokens_price_buy = self.tokens_price_now
        else:
            self.cryptos[0].get_btc_price_now(time = self.time_now)
            btc_price_now = self.cryptos[0].btc_price_now

            for crypto in self.cryptos:
                crypto.get_now(
                    action=self.identifier + 'price_update_' + self.action,
                    time = self.time_now,
                    btc_price = btc_price_now,
                    purchassed=False
                )
        
    def get_delta_time(self, initial=False):
        '''
        Get time deltas and ratios
        ''' 
        if initial:
            self.ratio_time_since_buy = 0
            self.days_since_buy = 0
    
        self.delta_time_since_buy = (self.time_now - self.time_buy).total_seconds()
        self.delta_since_last_time = (self.time_now - self.last_time).total_seconds()
        self.delta_time_since_claim = (self.time_now - self.last_claim_time).total_seconds()
        
        self.ratio_time_since_buy = self.delta_time_since_buy/(365*24*60*60)
        self.ratio_since_last_time = self.delta_since_last_time/(365*24*60*60)
        self.ratio_since_claim = self.delta_time_since_claim/(365*24*60*60)
        
        self.days_since_buy = self.delta_time_since_buy/(24*60*60)
        self.days_since_claim = self.delta_time_since_claim/(24*60*60)
    
    def get_quantity_APY_return(self, initial=False, manual=False):
        '''
        Calculates the estimated quantites increasse with the APY
        ''' 
        self.token_quantity_apy_return = []
        self.token_value_apy_return = []
        
        if initial:
            self.value_apy_return = 0
            self.delta_since_last_time = 0
            self.ratio_since_last_time = 0
            for crypto in self.cryptos:
                self.token_quantity_apy_return.append(0)
                self.token_value_apy_return.append(0)
        else:
            
            for crypto in self.cryptos:
                
                if not manual:
                    token_quantity_apy_return = crypto.quantity_now * (self.APY * self.ratio_since_claim)
                else:
                    token_quantity_apy_return = float(input('Enter {} tokens quantity pool rewards manually'.format(crypto.name)))

                self.token_quantity_apy_return.append(token_quantity_apy_return)
                self.token_value_apy_return.append(token_quantity_apy_return * crypto.price_now)
            
            self.value_apy_return = sum(self.token_value_apy_return)
            self.last_time = self.time_now
            
            if manual: self.update_APY(self.value_apy_return / (self.value_last_claim * self.ratio_since_claim) * 100)
                
    def get_quantity_now(self, initial=False, manual=False):
        '''
        Updates the quantity of each token of the liquidity pool due to the price variations. (Max 5 cryptos pool)
        '''
        if manual:
            manual = (input('Do you want to enter token quantities manually? (y/n)') == 'y')

        self.quantity_now = []
        if initial:
            self.quantity_buy = []
            for crypto in self.cryptos:
                self.quantity_now.append(crypto.quantity_now)
                self.quantity_buy.append(crypto.quantity_now)
        else:
            if not manual:
                num_cryptos = len(self.cryptos)

                if num_cryptos == 2:
                    new_quantities = self.function_2_cryptos()
                elif num_cryptos == 3:
                    new_quantities = self.function_3_cryptos()
                elif num_cryptos == 4:
                    new_quantities = self.function_4_cryptos()
                elif num_cryptos == 5:
                    new_quantities = self.function_5_cryptos()

                for quantity in new_quantities:
                    self.quantity_now.append(quantity)
            else:
                for crypto in self.cryptos:
                    quantity = float(input('Enter {} tokens quantity manually'.format(crypto.name)))
                    self.quantity_now.append(quantity)
                
    def udpdate_cryptos_quantities(self, initial=False):
        '''
        Updates all cryptos with new quantities
        '''
        if not initial:
            for crypto, quantity in zip(self.cryptos, self.quantity_now):
                # Update quantity but keep price and time. As these tokens are not purchassed or sold we want crypto.price_mean value to keep constant
                crypto.get_now(
                    quantity = quantity,
                    time=self.time_now,
                    price=crypto.price_now,
                    action=self.identifier+'quantity_update',
                    btc_price=crypto.btc_price_now,
                    purchassed=False
                )
    
    def get_value_now(self, initial=False):
        '''
        Gets the value of the liquidity pool
        '''
        self.value_now = 0
        self.tokens_value_now = []
        self.tokens_price_now = []
        for crypto, token_value_apy_return in zip(self.cryptos, self.token_value_apy_return):
            self.value_now = self.value_now + crypto.value_now
            self.tokens_value_now.append(crypto.value_now)
            self.tokens_price_now.append(crypto.price_now)
            
        if initial: 
            self.value_buy = self.value_now
            self.value_last_claim = self.value_now
            self.tokens_value_buy = self.tokens_value_now
            self.tokens_price_buy = self.tokens_price_now
    
    def get_btc_value_now(self, initial=False):
        '''
        Gets the value of the liquidity pool in BTC
        '''
        self.btc_value_now = 0
        for crypto, token_value_apy_return in zip(self.cryptos, self.token_value_apy_return):
            self.btc_value_now = self.btc_value_now + crypto.btc_value_now + token_value_apy_return / crypto.btc_price_now
        
        if initial: self.btc_value_buy = self.btc_value_now
    
    def get_impermanent_loss(self, initial=False):
        '''
        Calculates the impermanent loss of the liquidity pool
        '''
        if initial: 
            self.impermanent_loss = 0
            self.impermanent_value = 0
            self.impermanent_token_value = []
            for crypto in self.cryptos:
                self.impermanent_token_value.append(crypto.value_now)
        else:
            self.impermanent_token_value = []
            for quantity_buy, crypto in zip(self.quantity_buy, self.cryptos):
                impermanent_token_value = quantity_buy * crypto.price_now
                self.impermanent_token_value.append(impermanent_token_value)
                
            self.impermanent_value = sum(self.impermanent_token_value)
                
            self.impermanent_loss = self.impermanent_value - (self.value_now + self.value_apy_return)
        
    def get_impermanent_loss_percentage(self, initial=False):
        '''
        Calculates the impermanent loss of the liquidity pool
        '''
        if initial: 
            self.impermanent_loss_percentage = 0
        else:
            self.impermanent_loss_percentage = round(100 * self.impermanent_loss / self.impermanent_value,1)
            
    def get_return_total(self, initial):
        '''
        Calculates the return of the liquidity pool
        '''
        if initial:
            self.return_total = 0
            self.equivalent_apy_1y = 0
        else:
            self.return_total = round(100 * (self.value_now + self.value_apy_return - self.value_buy)/self.value_buy,1)
            self.equivalent_apy_1y = self.return_total / self.ratio_time_since_buy
    
    def get_tokens_proportion(self, initial=False):
        '''
        Calculates the proportion of each token at the begininig of the liquidity pool
        ''' 
        self.token = []
        token_proportion = []
        for crypto in self.cryptos:
            crypto.get_token_proportion(crypto.value_now / self.value_now)
            self.token.append(crypto.symbol)
            token_proportion.append(round(crypto.token_proportion * 100, 1))
        
        self.token_proportion = token_proportion
    
    def update_APY(self, APY=''):
        '''
        Updates APY value
        '''
        self.APY = float(APY)/100
        
    def claim_rewards(self, time=''):
        '''
        Claim rewards and add them
        '''
        self.get_time_now(time = time)
        self.last_claim_time= self.time_now
        self.cryptos[0].get_btc_price_now(time=time)
        self.quantity_now = []
        
        for reward, crypto in zip(self.token_quantity_apy_return, self.cryptos):
            # Update quantity but keep price and time. As these tokens are not purchassed or sold we want crypto.price_mean value to keep constant
            crypto.get_now(
                quantity = crypto.quantity_now + reward,
                time=self.time_now,
                action=self.identifier+'claim_rewards',
                btc_price = self.cryptos[0].btc_price_now,
                purchassed=False
            )
            
            self.quantity_now.append(crypto.quantity_now)   
            
            # Reset APY returns
            self.token_quantity_apy_return = 0
        
        self.get_now(time=self.last_claim_time)
        
        self.value_last_claim = self.value_now
        
    def get_identifier(self, initial=False, identifier=''):
        '''
        Gets an identifier for this LP
        '''
        if initial:
            if identifier != '':
                self.identifier = identifier
            else:
                identifier = ''
                for crypto in self.cryptos:
                    identifier = identifier + crypto.symbol + '_' + str(int(round(100 * crypto.token_proportion, 0))) + '_'
            self.identifier = 'LP_' + identifier
            
    def round_floats(self, lst):
        return list(map(lambda x: round(x,6), lst))
            
    def get_info(self):
        '''
        Outputs a dictionary of all values inside the class
        '''
        dict_info = {
            'time_buy': self.time_buy,
            'time_now': self.time_now,
            'last_time': self.last_time,
            'last_claim_time': self.last_claim_time,
            
            #'delta_since_last_time': self.delta_since_last_time,
            #'delta_time_since_buy': self.delta_time_since_buy,
            #'delta_time_since_claim': self.delta_time_since_claim,
            
            #'ratio_since_last_time': self.ratio_since_last_time,
            #'ratio_time_since_buy': self.ratio_time_since_buy,
            #'ratio_since_claim': self.ratio_since_claim,
            
            'days_since_buy': self.days_since_buy,
            'days_since_claim': self.days_since_claim,
            
            'quantity_buy': self.round_floats(self.quantity_buy),
            'quantity_now': self.round_floats(self.quantity_now),
            'quantity_apy_return': self.round_floats(self.token_quantity_apy_return),
            'APY %': self.APY,
            
            'tokens_price_buy': self.round_floats(self.tokens_price_buy),
            'tokens_price_now': self.round_floats(self.tokens_price_now),
            
            'tokens_value_buy': self.round_floats(self.tokens_value_buy),
            'tokens_value_now': self.round_floats(self.tokens_value_now),
            'token_value_apy_return': self.round_floats(self.token_value_apy_return),
            
            'impermanent_token_value': self.round_floats(self.impermanent_token_value),
            'value_buy': self.value_buy,
            'value_now': self.value_now,
            'value_apy_return': self.value_apy_return,
            'total_value_now': self.value_now + self.value_apy_return,
            
            'impermanent_value': self.impermanent_value,
            'impermanent_loss': self.impermanent_loss,
            
            'impermanent_loss_percentage %': self.impermanent_loss_percentage,
            'return_total %': self.return_total,
            'equivalent_apy_1y %': self.equivalent_apy_1y,
            
            'btc_value_buy': self.btc_value_buy,
            'btc_value_now': self.btc_value_now,
            'btc_return %': round((self.btc_value_now - self.btc_value_buy) / self.btc_value_buy * 100, 2),
            
            'token': self.token,
            'token_proportion': self.round_floats(self.token_proportion),
            'k': self.k
        }
        self.info = dict_info
        
    def get_constants(self):
        '''
        Calculates constants needed for the calculation of liquidity pools token quantity
        
        Exemple for i,j tokens:
        Civj = pi / ((1-pi)*vi) * vj
        '''
        # Calculate C*v* constants
        Cv = []
        for crypto in self.cryptos:
            C = crypto.token_proportion / ((1 - crypto.token_proportion) * crypto.price_now)
            c = []
            for crypto_2 in self.cryptos:
                c.append(C * crypto_2.price_now)
            Cv.append(c)
        
        return Cv   
        
    def function_2_cryptos(self):
        '''
        Function for liquidity pool for 2 cryptos
        
        From:
            q1 * v1 / (q1 * v1 + q2 * v2) = p1
            q1 * q2 = K
        
        Final equations:
              q1 - q2 * C1v2 = 0
              q1 * q2 - K = 0
        '''
        Cv = self.get_constants()
            
        q1, q2 = symbols('q1 q2')
        
        F = [
            q1 - q2 * Cv[0][1],
            q1 * q2 - self.k
        ]
        
        solutions = solve_poly_system(F, q1, q2)
        return self.select_solution(solutions)
    
    def function_3_cryptos(self):
        '''
        Function for liquidity pool for 3 cryptos
        
        Final equations:
              q1        - q2 * C1v2 - q3 * C1v3 = 0
            - q1 * C2v1 + q2        - q3 * C2v3 = 0
              q1 * q2 * q3 - K = 0
        '''
        Cv = self.get_constants()
            
        q1, q2, q3 = symbols('q1 q2 q3')
        
        F = [
              q1            - q2 * Cv[0][1] - q3 * Cv[0][2],
            - q1 * Cv[1][0] + q2            - q3 * Cv[1][2],
            q1 * q2 * q3 - self.k
        ]
        
        solutions = solve_poly_system(F, q1, q2, q3)
        return self.select_solution(solutions)
    
    def function_4_cryptos(self):
        '''
        Function for liquidity pool for 4 cryptos
        
        Final equations:
              q1        - q2 * C1v2 - q3 * C1v3 - q4 * C1v4 = 0
            - q1 * C2v1 + q2        - q3 * C2v3 - q4 * C1v4 = 0
            - q1 * C2v1 - q2 * C1v2 + q3        - q4 * C1v4 = 0
              q1 * q2 * q3 * q4 - K = 0
        '''
        Cv = self.get_constants()
            
        q1, q2, q3, q4 = symbols('q1 q2 q3 q4')
        
        F = [
              q1            - q2 * Cv[0][1] - q3 * Cv[0][2] - q4 * Cv[0][3],
            - q1 * Cv[1][0] + q2            - q3 * Cv[1][2] - q4 * Cv[1][3],
            - q1 * Cv[2][0] - q2 * Cv[2][1] + q3            - q4 * Cv[2][3],
            q1 * q2 * q3 * q4 - self.k
        ]
        
        solutions = solve_poly_system(F, q1, q2, q3, q4)
        return self.select_solution(solutions)
    
    def function_5_cryptos(self):
        '''
        Function for liquidity pool for 5 cryptos
        
        Final equations:
              q1        - q2 * C1v2 - q3 * C1v3 - q4 * C1v4 - q5 * C1v5 = 0
            - q1 * C2v1 + q2        - q3 * C2v3 - q4 * C2v4 - q5 * C2v5 = 0
            - q1 * C3v1 - q2 * C3v2 + q3        - q4 * C3v4 - q5 * C3v5 = 0
            - q1 * C4v1 - q2 * C4v2 - q3 * C4v3 + q4        - q5 * C4v5 = 0
              q1 * q2 * q3 * q4 - K = 0
        '''
        Cv = self.get_constants()
            
        q1, q2, q3, q4, q5 = symbols('q1 q2 q3 q4 q5')
        
        F = [
              q1            - q2 * Cv[0][1] - q3 * Cv[0][2] - q4 * Cv[0][3] - q5 * Cv[0][4],
            - q1 * Cv[1][0] + q2            - q3 * Cv[1][2] - q4 * Cv[1][3] - q5 * Cv[1][4],
            - q1 * Cv[2][0] - q2 * Cv[2][1] + q3            - q4 * Cv[2][3] - q5 * Cv[2][4],
            - q1 * Cv[3][0] - q2 * Cv[3][1] - q3 * Cv[3][2] + q4            - q5 * Cv[3][4],
            q1 * q2 * q3 * q4 * q5 - self.k
        ]
        
        solutions = solve_poly_system(F, q1, q2, q3, q4, q5)
        return self.select_solution(solutions)
    
    def select_solution(self, solutions):
        '''
        Selects coherent solution from polynomial solution calculation on get_quantity_now_X methods
        '''
        sol = []
        for solution in solutions:
            for q in solution:
                if isinstance(q, sympy.Float) and q > 0:
                    sol.append(q)
                else:
                    break
        return sol
    
    def get_trace(self, initial=False):
        '''
        Keeps trace of the evolution of the crypto
        '''
        action = 'lp_' + self.action
        
        if initial:
            self.trace = {}
            self.trace[self.identifier] = {}
            action = 'lp_start'
            
        self.timestamp = self.time_now.strftime("%Y/%m/%d %H:%M:%S")
        
        self.trace[self.identifier][self.counter] = {}
        self.trace[self.identifier][self.counter]['state'] = self.info
        self.trace[self.identifier][self.counter]['action'] = action

In [21]:
from math import sqrt

def UniSwapV3(LiquidityPool):
    def __init__(self, crypto_x, crypto_y, top_range, bottom_range, APY=0, time=''):
        '''
        Initialisation of the liquidity pool in UniSwapV3
        '''
        super().__init__(length, length)
        self.crypto_x = crypto_x
        self.crypto_y = crypto_y
        self.top_range = top_range
        # self.botom_range = bottom_range
        self.APY = APY
        self.crypto_x_price_buy = crypto_x.price_now
        self.crypto_y_price_buy = crypto_y.price_now
        self.crypto_x_quantity_buy = crypto_x.quantity_now
        self.crypto_y_quantity_buy = crypto_y.quantity_now
    
    def get_relative_price_buy(self):
        '''
        Calculates relative prices between assets at LP buy price
        '''
        self.relative_price_buy = self.crypto_x_price_buy / self.crypto_y_price_buy
    
    def get_relative_price_now(self):
        '''
        Calculates relative prices between assets at present price
        '''
        self.relative_price_now = crypto_x.price_now / crypto_y.price_now
        
    def get_liquidity(self):
        '''
        Calculates the liquidity of the positons
        '''
        P = self.relative_price_buy
        x = self.crypto_x_quantity_buy
        y = self.crypto_y_quantity_buy
        pb = self.top_range
        pa = self.bottom_range
        
        l_x = x * sqrt(P) * sqrt(pb) / (sqrt(pb) - sqrt(P))
        l_y = y / (sqrt(P) - sqrt(pa))
        
        self.liquidity = min(l_x, l_y)
        
    def get_quantity_now_v3(self):
        '''
        Calculates new cryptos quantities after prices updates
        '''
        Pi = self.relative_price_now
        L = self.liquidity
        pb = self.top_range
        pa = self.bottom_range
        
        xi = L * (sqrt(pb) - sqrt(Pi)) / (sqrt(Pi) * sqrt(pb))
        yi = L * (sqrt(Pi) - sqrt(pa))
        
        

In [22]:
LP = LiquidityPool([btc,eth,matic], APY=60, time='11/03/2022 12:00')
LP.trace

{'LP_btc_27_eth_42_matic_30_': {0: {'state': {'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'last_time': datetime.datetime(2022, 3, 11, 12, 0),
    'last_claim_time': datetime.datetime(2022, 3, 11, 12, 0),
    'days_since_buy': 0.0,
    'days_since_claim': 0.0,
    'quantity_buy': [0.001, 0.0232, 30],
    'quantity_now': [0.001, 0.0232, 30],
    'quantity_apy_return': [0, 0, 0],
    'APY %': 0.6,
    'tokens_price_buy': [39010.026667, 2597.49, 1.44148],
    'tokens_price_now': [39010.026667, 2597.49, 1.44148],
    'tokens_value_buy': [39.010027, 60.261768, 43.2444],
    'tokens_value_now': [39.010027, 60.261768, 43.2444],
    'token_value_apy_return': [0, 0, 0],
    'impermanent_token_value': [39.010027, 60.261768, 43.2444],
    'value_buy': 142.51619469999997,
    'value_now': 142.51619469999997,
    'value_apy_return': 0,
    'total_value_now': 142.51619469999997,
    'impermanent_value': 0,
    'impermanent_loss': 0,
  

In [23]:
LP.get_now(time='10/12/2022 22:00')
LP.trace

BTC/USDT 10/12/2022 22:00 lbank2 17173.08
BTC/USDT 10/12/2022 22:00 huobipro 17169.93
BTC/USDT 10/12/2022 22:00 hollaex 17176.71
price BTC/USDT 10/12/2022 22:00 17173.24
BTC/USDT 10/12/2022 22:00 lbank2 17173.08
BTC/USDT 10/12/2022 22:00 huobipro 17169.93
BTC/USDT 10/12/2022 22:00 hollaex 17176.71
price BTC/USDT 10/12/2022 22:00 17173.24
ETH/USDT 10/12/2022 22:00 lbank2 1272.07
ETH/USDT 10/12/2022 22:00 huobipro 1272.7
ETH/USDT 10/12/2022 22:00 hollaex 1272.66
price ETH/USDT 10/12/2022 22:00 1272.4766666666667
MATIC/USDT 10/12/2022 22:00 lbank2 0.91
MATIC/USDT 10/12/2022 22:00 huobipro 0.910209
MATIC/USDT NOT FOUND in  hollaex
MATIC/USDT 10/12/2022 22:00 bitmart 0.9104
price MATIC/USDT 10/12/2022 22:00 0.9102030000000001


{'LP_btc_27_eth_42_matic_30_': {0: {'state': {'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'last_time': datetime.datetime(2022, 3, 11, 12, 0),
    'last_claim_time': datetime.datetime(2022, 3, 11, 12, 0),
    'days_since_buy': 0.0,
    'days_since_claim': 0.0,
    'quantity_buy': [0.001, 0.0232, 30],
    'quantity_now': [0.001, 0.0232, 30],
    'quantity_apy_return': [0, 0, 0],
    'APY %': 0.6,
    'tokens_price_buy': [39010.026667, 2597.49, 1.44148],
    'tokens_price_now': [39010.026667, 2597.49, 1.44148],
    'tokens_value_buy': [39.010027, 60.261768, 43.2444],
    'tokens_value_now': [39.010027, 60.261768, 43.2444],
    'token_value_apy_return': [0, 0, 0],
    'impermanent_token_value': [39.010027, 60.261768, 43.2444],
    'value_buy': 142.51619469999997,
    'value_now': 142.51619469999997,
    'value_apy_return': 0,
    'total_value_now': 142.51619469999997,
    'impermanent_value': 0,
    'impermanent_loss': 0,
  

In [24]:
LP.get_now(manual=True)
LP.trace

BTC/USDT 03/03/2023 13:52 lbank2 22351.13
BTC/USDT 03/03/2023 13:52 huobipro 22353.47
BTC/USDT NOT FOUND in  hollaex
BTC/USDT 03/03/2023 13:52 bitmart 22351.7
price BTC/USDT 03/03/2023 13:52 22352.100000000002
BTC/USDT 03/03/2023 13:52 lbank2 22351.58
BTC/USDT 03/03/2023 13:52 huobipro 22352.42
BTC/USDT 03/03/2023 13:52 hollaex 22362.99
price BTC/USDT 03/03/2023 13:52 22355.663333333334
ETH/USDT 03/03/2023 13:52 lbank2 1565.6
ETH/USDT 03/03/2023 13:52 huobipro 1565.48
ETH/USDT 03/03/2023 13:52 hollaex 1568.75
price ETH/USDT 03/03/2023 13:52 1566.61
MATIC/USDT 03/03/2023 13:52 lbank2 1.1646
MATIC/USDT 03/03/2023 13:52 huobipro 1.165124
MATIC/USDT NOT FOUND in  hollaex
MATIC/USDT 03/03/2023 13:52 bitmart 1.1653
price MATIC/USDT 03/03/2023 13:52 1.165008


Enter bitcoin tokens quantity pool rewards manually 1
Enter ethereum tokens quantity pool rewards manually 1
Enter polygon tokens quantity pool rewards manually 1
Do you want to enter token quantities manually? (y/n) n


{'LP_btc_27_eth_42_matic_30_': {0: {'state': {'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'last_time': datetime.datetime(2022, 3, 11, 12, 0),
    'last_claim_time': datetime.datetime(2022, 3, 11, 12, 0),
    'days_since_buy': 0.0,
    'days_since_claim': 0.0,
    'quantity_buy': [0.001, 0.0232, 30],
    'quantity_now': [0.001, 0.0232, 30],
    'quantity_apy_return': [0, 0, 0],
    'APY %': 0.6,
    'tokens_price_buy': [39010.026667, 2597.49, 1.44148],
    'tokens_price_now': [39010.026667, 2597.49, 1.44148],
    'tokens_value_buy': [39.010027, 60.261768, 43.2444],
    'tokens_value_now': [39.010027, 60.261768, 43.2444],
    'token_value_apy_return': [0, 0, 0],
    'impermanent_token_value': [39.010027, 60.261768, 43.2444],
    'value_buy': 142.51619469999997,
    'value_now': 142.51619469999997,
    'value_apy_return': 0,
    'total_value_now': 142.51619469999997,
    'impermanent_value': 0,
    'impermanent_loss': 0,
  

In [25]:
LP.claim_rewards()

BTC/USDT 03/03/2023 13:52 lbank2 22351.59
BTC/USDT 03/03/2023 13:52 huobipro 22352.65
BTC/USDT 03/03/2023 13:52 hollaex 22362.99
price BTC/USDT 03/03/2023 13:52 22355.743333333336
BTC/USDT 03/03/2023 13:52 lbank2 22351.59
BTC/USDT 03/03/2023 13:52 huobipro 22352.65
BTC/USDT 03/03/2023 13:52 hollaex 22362.99
price BTC/USDT 03/03/2023 13:52 22355.743333333336
ETH/USDT 03/03/2023 13:52 lbank2 1565.31
ETH/USDT 03/03/2023 13:52 huobipro 1565.84
ETH/USDT 03/03/2023 13:52 hollaex 1568.75
price ETH/USDT 03/03/2023 13:52 1566.6333333333332
MATIC/USDT 03/03/2023 13:52 lbank2 1.1643
MATIC/USDT 03/03/2023 13:52 huobipro 1.165266
MATIC/USDT 03/03/2023 13:52 hollaex 1.16541
price MATIC/USDT 03/03/2023 13:52 1.164992
BTC/USDT 03/03/2023 13:52 lbank2 22355.06
BTC/USDT 03/03/2023 13:52 huobipro 22357.0
BTC/USDT 03/03/2023 13:52 hollaex 22362.99
price BTC/USDT 03/03/2023 13:52 22358.350000000002
BTC/USDT 03/03/2023 13:52 lbank2 22355.06
BTC/USDT 03/03/2023 13:52 huobipro 22357.0
BTC/USDT 03/03/2023 13:5

In [26]:
LP.trace

{'LP_btc_27_eth_42_matic_30_': {0: {'state': {'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'last_time': datetime.datetime(2022, 3, 11, 12, 0),
    'last_claim_time': datetime.datetime(2022, 3, 11, 12, 0),
    'days_since_buy': 0.0,
    'days_since_claim': 0.0,
    'quantity_buy': [0.001, 0.0232, 30],
    'quantity_now': [0.001, 0.0232, 30],
    'quantity_apy_return': [0, 0, 0],
    'APY %': 0.6,
    'tokens_price_buy': [39010.026667, 2597.49, 1.44148],
    'tokens_price_now': [39010.026667, 2597.49, 1.44148],
    'tokens_value_buy': [39.010027, 60.261768, 43.2444],
    'tokens_value_now': [39.010027, 60.261768, 43.2444],
    'token_value_apy_return': [0, 0, 0],
    'impermanent_token_value': [39.010027, 60.261768, 43.2444],
    'value_buy': 142.51619469999997,
    'value_now': 142.51619469999997,
    'value_apy_return': 0,
    'total_value_now': 142.51619469999997,
    'impermanent_value': 0,
    'impermanent_loss': 0,
  

In [27]:
eth.trace

{'eth_in_polygon_wallet_polygon_2022_3_11_12_0': {'id': 'ethereum',
  'name': 'ethereum',
  'categories': ['Smart Contract Platform',
   'Ethereum Ecosystem',
   'Layer 1 (L1)'],
  'chain': 'polygon',
  'location': 'wallet_polygon',
  'symbol': 'eth',
  0: {'state': {'purchassed': True,
    'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'time_delta': datetime.timedelta(0),
    'price_old': 0,
    'price_now': 2597.49,
    'price_mean_old': 0,
    'price_mean': 2597.49,
    'fees': 0,
    'fees_cumulated': 0,
    'quantity_old': 0,
    'quantity_now': 0.0232,
    'quantity_delta': 0.0232,
    'value_now': 60.261768,
    'value_mean': 60.26176799999999,
    'fees_cumulated_value': 0,
    'btc_price_old': 0,
    'btc_price_now': 39010.026666666665,
    'btc_price_mean': 39010.0266667,
    'btc_value_now': 0.0015448,
    'btc_value_mean': 0.0015448,
    'token_proportion': 1,
    'return %': 0.0,
    'btc_return %': 0.0},
   'a

In [28]:
btc.trace

{'btc_in_polygon_wallet_polygon_2022_3_11_12_0': {'id': 'bitcoin',
  'name': 'bitcoin',
  'categories': ['Cryptocurrency', 'Layer 1 (L1)'],
  'chain': 'polygon',
  'location': 'wallet_polygon',
  'symbol': 'btc',
  0: {'state': {'purchassed': True,
    'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'time_delta': datetime.timedelta(0),
    'price_old': 0,
    'price_now': 39010.026666666665,
    'price_mean_old': 0,
    'price_mean': 39010.0266667,
    'fees': 0,
    'fees_cumulated': 0,
    'quantity_old': 0,
    'quantity_now': 0.001,
    'quantity_delta': 0.001,
    'value_now': 39.0100267,
    'value_mean': 39.01002666670001,
    'fees_cumulated_value': 0,
    'btc_price_old': 0,
    'btc_price_now': 39010.026666666665,
    'btc_price_mean': 39010.0266667,
    'btc_value_now': 0.001,
    'btc_value_mean': 0.001,
    'token_proportion': 1,
    'return %': 0.0,
    'btc_return %': 0.0},
   'action': 'crypto_initial'},
  1:

In [ ]:
import pickle

pfs = pf
fileObject = open('Portfolio.obj', 'wb') 
pickle.dump(pfs, fileObject)

In [55]:
fileObject = open('Portfolio.obj', 'rb') 
som = pickle.load(fileObject)

In [56]:
vars(som)

{'name': 'portfolio',
 'wallets': {'polygon': <__main__.Wallet at 0x1f77b2345b0>,
  'arbitrum': <__main__.Wallet at 0x1f77c4a2f70>}}

In [57]:
som.wallets['polygon'].get_info()
som.wallets['polygon'].info

{'bitcoin': {'purchassed': False,
  'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
  'time_now': datetime.datetime(2023, 3, 3, 11, 47, 16, 563619),
  'time_delta': datetime.timedelta(days=356, seconds=85636, microseconds=563619),
  'price_old': 22371.420000000002,
  'price_now': 22371.420000000002,
  'price_mean_old': 39010.0266667,
  'price_mean': 39010.0266667,
  'fees': 0,
  'fees_cumulated': 0,
  'quantity_old': 0.0381220606576562,
  'quantity_now': 0.0381220606576562,
  'quantity_delta': -0.963021931361836,
  'value_now': 852.8446302,
  'value_mean': 1487.14260284472,
  'fees_cumulated_value': 0,
  'btc_price_old': 22373.27,
  'btc_price_now': 22373.27,
  'btc_price_mean': 39010.0266667,
  'btc_value_now': 0.0381189,
  'btc_value_mean': 0.0381221,
  'token_proportion': 0.27372346589,
  'return %': -42.7,
  'btc_return %': 0.0},
 'wrapped bitcoin': {'purchassed': True,
  'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
  'time_now': datetime.datetime(2022, 3, 11, 12, 0),
  't

In [58]:
def split_crypto(old_crypto, proportion=0, quantity=0):
    '''
    Splits (by proportion or quantity) one Crypto object into two Crypto objects
    '''
    if proportion != 0:
        new_crypto_quantity_now = old_crypto.quantity_now * proportion
        old_crypto_quantity_now = old_crypto.quantity_now * (1 - proportion)
        
        split = 'split_' + str(proportion) + '_proportion'
        
    elif quantity != 0:
        if old_crypto.quantity_now < quantity:
            print('Error: quantity is higher than available')
            return
        new_crypto_quantity_now = quantity
        old_crypto_quantity_now = old_crypto.quantity_now - quantity
        
        split = 'split_' + str(quantity) + '_quantity'
    else:
        print('Error: You should fill proportion or qunatity')
        return

    new_crypto = Crypto(name = old_crypto.name,
                        symbol = old_crypto.symbol, 
                        quantity = new_crypto_quantity_now,
                        time = old_crypto.time_now,
                        price = old_crypto.price_now, 
                        btc_price = old_crypto.btc_price_now, 
                        wallet = old_crypto.wallet)

    new_crypto.get_price_mean(price = old_crypto.price_mean)
    new_crypto.get_now()

    new_crypto.get_btc_price_mean(price = old_crypto.btc_price_mean)

    #old_crypto.get_quantity_now(old_crypto_quantity_now)
    old_crypto.get_now(quantity = old_crypto_quantity_now, action = split, purchassed = False)

    return old_crypto, new_crypto

In [59]:
eth_old, eth_new = split_crypto(eth, quantity = 0.001)

ETH/USDT 03/03/2023 11:48 huobipro 1567.08
ETH/USDT 03/03/2023 11:48 lbank2 1567.01
ETH/USDT NOT FOUND in  hollaex
ETH/USDT 03/03/2023 11:48 bitmart 1566.78
price ETH/USDT 03/03/2023 11:48 1566.9566666666667
BTC/USDT 03/03/2023 11:48 huobipro 22368.29
BTC/USDT 03/03/2023 11:48 lbank2 22368.52
BTC/USDT NOT FOUND in  hollaex
BTC/USDT 03/03/2023 11:48 bitmart 22368.39
price BTC/USDT 03/03/2023 11:48 22368.399999999998
ETH/USDT 03/03/2023 11:48 huobipro 1567.22
ETH/USDT 03/03/2023 11:48 lbank2 1567.01
ETH/USDT NOT FOUND in  hollaex
ETH/USDT 03/03/2023 11:48 bitmart 1566.78
price ETH/USDT 03/03/2023 11:48 1567.0033333333333
BTC/USDT 03/03/2023 11:48 huobipro 22366.14
BTC/USDT 03/03/2023 11:48 lbank2 22368.01
BTC/USDT NOT FOUND in  hollaex
BTC/USDT 03/03/2023 11:48 bitmart 22368.39
price BTC/USDT 03/03/2023 11:48 22367.513333333332


In [60]:
eth_new.trace

{'eth_in_polygon_wallet_polygon_2023_3_3_11_47': {'id': 'ethereum',
  'name': 'ethereum',
  'categories': ['Smart Contract Platform',
   'Ethereum Ecosystem',
   'Layer 1 (L1)'],
  'chain': 'polygon',
  'location': 'wallet_polygon',
  'symbol': 'eth',
  0: {'state': {'purchassed': True,
    'time_buy': datetime.datetime(2023, 3, 3, 11, 47, 16, 563619),
    'time_now': datetime.datetime(2023, 3, 3, 11, 47, 16, 563619),
    'time_delta': datetime.timedelta(0),
    'price_old': 0,
    'price_now': 1567.1466666666668,
    'price_mean_old': 0,
    'price_mean': 1567.1466667,
    'fees': 0,
    'fees_cumulated': 0,
    'quantity_old': 0,
    'quantity_now': 0.001,
    'quantity_delta': 0.001,
    'value_now': 1.5671467,
    'value_mean': 1.5671466667,
    'fees_cumulated_value': 0,
    'btc_price_old': 0,
    'btc_price_now': 22373.27,
    'btc_price_mean': 22373.27,
    'btc_value_now': 7e-05,
    'btc_value_mean': 7e-05,
    'token_proportion': 1,
    'return %': 0.0,
    'btc_return %': 0

In [61]:
eth_old.trace

{'eth_in_polygon_wallet_polygon_2022_3_11_12_0': {'id': 'ethereum',
  'name': 'ethereum',
  'categories': ['Smart Contract Platform',
   'Ethereum Ecosystem',
   'Layer 1 (L1)'],
  'chain': 'polygon',
  'location': 'wallet_polygon',
  'symbol': 'eth',
  0: {'state': {'purchassed': True,
    'time_buy': datetime.datetime(2022, 3, 11, 12, 0),
    'time_now': datetime.datetime(2022, 3, 11, 12, 0),
    'time_delta': datetime.timedelta(0),
    'price_old': 0,
    'price_now': 2597.49,
    'price_mean_old': 0,
    'price_mean': 2597.49,
    'fees': 0,
    'fees_cumulated': 0,
    'quantity_old': 0,
    'quantity_now': 0.0232,
    'quantity_delta': 0.0232,
    'value_now': 60.261768,
    'value_mean': 60.26176799999999,
    'fees_cumulated_value': 0,
    'btc_price_old': 0,
    'btc_price_now': 39010.026666666665,
    'btc_price_mean': 39010.0266667,
    'btc_value_now': 0.0015448,
    'btc_value_mean': 0.0015448,
    'token_proportion': 1,
    'return %': 0.0,
    'btc_return %': 0.0},
   'a

In [136]:
def swap(crypto, new_name, new_quantity, gas_crypto, gas_quantity=0):
    '''
    Allows to change a cryptocurrency by another
    '''
    crypto_value = crypto.value_now
    
    new_crypto = Crypto(name=new_name, quantity=new_quantity, time=crypto.time_now, chain=crypto.chain, location=crypto.location)
    new_crypto_value = new_crypto.value_now
    
    crypto.close()
    
    gas_crypto.get_fees(gas_quantity)
    
    variation =  abs(crypto_value - new_crypto_value)/crypto_value * 100
    
    if variation > 10:
        print('Check quantities swaped as variation is', variation, '%')
    
    return new_crypto
    

In [137]:
#def split_swap(cryptos, new_names, new_quantities, gas_quantity=0):

cryptos = [btc]
old_quantity_variation =[0.0007]
new_names = ['Bitcoin','Ethereum','Polygon']
new_quantities = [0.0005, 0.0001, 0.01]

cryptos_names = []
cryptos_dict = {}
for crypto in cryptos:
    cryptos_names.append(crypto.name)
    time = crypto.time_now
    btc_price = crypto.btc_price_now
    wallet = crypto.wallet
    cryptos_dict[crypto.name] = crypto
    

old_cryptos = []
new_cryptos = []
for name, quantity in zip(new_names, new_quantities):
    if name.lower() in cryptos_names:
        print(name)
        old_crypto, new_crypto = split_crypto(cryptos_dict[name.lower()], quantity=quantity)
        old_cryptos.append(old_crypto)
        new_cryptos.append(new_crypto)
    else:
        new_crypto = Crypto(name=name, quantity=quantity, time=time, btc_price=btc_price, wallet=wallet)
        new_cryptos.append(new_crypto)
                

Bitcoin
BTC/USDT 27/02/2023 23:23 huobipro 23371.49
BTC/USDT 27/02/2023 23:23 lbank2 23372.01
BTC/USDT NOT FOUND in  hollaex
BTC/USDT 27/02/2023 23:23 bitmart 23400.96
price BTC/USDT 27/02/2023 23:23 23381.486666666664
BTC/USDT 27/02/2023 23:23 huobipro 23378.66
BTC/USDT 27/02/2023 23:23 lbank2 23371.53
BTC/USDT NOT FOUND in  hollaex
BTC/USDT 27/02/2023 23:23 bitmart 23375.39
price BTC/USDT 27/02/2023 23:23 23375.193333333333
BTC/USDT 27/02/2023 23:24 huobipro 23388.03
BTC/USDT 27/02/2023 23:24 lbank2 23385.55
BTC/USDT 27/02/2023 23:24 hollaex 23379.38
price BTC/USDT 27/02/2023 23:24 23384.320000000003
BTC/USDT 27/02/2023 23:24 huobipro 23393.65
BTC/USDT 27/02/2023 23:24 lbank2 23385.55
BTC/USDT 27/02/2023 23:24 hollaex 23379.38
price BTC/USDT 27/02/2023 23:24 23386.193333333333
ETH/USDT 27/02/2023 23:13 huobipro 1628.21
ETH/USDT 27/02/2023 23:13 lbank2 1628.36
ETH/USDT 27/02/2023 23:13 hollaex 1628.52
price ETH/USDT 27/02/2023 23:13 1628.3633333333335
MATIC/USDT 27/02/2023 23:13 huobi

In [138]:
cryptos_dict

{'bitcoin': <__main__.Crypto at 0x1f49ee69520>}

In [139]:
for crypto in old_cryptos:
    print(crypto.quantity_now)

0.00115867593555381


In [140]:
for crypto in new_cryptos:
    print(crypto.quantity_now)

0.0005
0.0001
0.01


In [ ]:
def launch_liquidity_pool(cryptos_list, proportions_list, crypto_gas, APY=0, gas=0):
    
    old_cryptos_list = []
    new_cryptos_list = []
    
    for crypto, proportion in zip(cryptos_list, proportions_list):
        old_cryptos, new_cryptos = split_crypto(crypto, proportion, location='liquidity_pool')
        
        old_cryptos_list.append(old_cryptos)
        new_cryptos_list.append(new_cryptos)
        
    return LiquidityPool(new_cryptos_list, APY=APY)


In [ ]:
def bridge(crypto, crypto_chain1, crypto_chain2, gas_chain1=0, gas_chain2=0, new_chain='Fantom'):
    '''
    Allows changing the chain of a cryptocurrency
    '''
    crypto.update_chain(new_chain)
    crypto_chain1.get_quantity_now(crypto_chain1.quantity_now - gas_chain1)
    crypto_chain2.get_quantity_now(crypto_chain2.quantity_now - gas_chain2)
    
    # Update cryptos
    crypto.get_now('bridge_from_' + crypto.chain + '_to_' + new_chain)
    
    for crypto in [crypto_chain2, crypto_chain2]:
        crypto.get_now('gas_bridge_from_' + crypto.chain + '_to_' + new_chain)

In [ ]:
# Retrieve a list of verified contracts
base_url = "https://polygonscan.com/tx/0xce000c0308a817d4c35022bdd257a874693e87925ecadeeee550bf90c3ab8a3a"
resp = requests.get(base_url)
sess = requests.Session()
BeautifulSoup(sess.get(base_url).text, 'html.parser')